Ethan Zhang
ez2262

In [ ]:
import scipy.io
mat = scipy.io.loadmat('digits.mat') 

In [ ]:
from sklearn.model_selection import train_test_split
X = mat['X']
Y = mat['Y']

# Make a train/test split using 20% test size
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 100)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
from scipy.stats import multivariate_normal
import numpy as np
means = np.zeros(shape=(10, 100))
counts = [0]*10
covariances = np.zeros(shape=(10, 100, 100))
for i in range(len(y_train)):
  # for each row in labels, get the specific label and update count of that label
  y_row = y_train[i]
  label = y_row[0]
  counts[label]+=1

  #Means
  x_row = X_train[i]
  for j in range(len(x_row)):
    means[label][j]+=x_row[j]

for i in range(len(counts)):
  means[i]/=counts[i]
#Covariances
for i in range(len(y_train)):
  y_row = y_train[i]
  label = y_row[0]
  x_row = X_train[i]

  x_rowarr = np.asarray(x_row)
  m = np.subtract(x_rowarr, means[label])[:,None]
  covariances[label] = covariances[label]+np.matmul(m, m.transpose())

for i in range(len(counts)):
  covariances[i]/=counts[i]



In [ ]:
correct = 0
total = 0
for i in range(len(X_test)):
  probs = [1]*10
  for k in range(10):
    probs[k]*=multivariate_normal.pdf(X_test[i], mean=means[k], cov=covariances[k])
  # print(np.argmax(probs), y_test[i][0])
  if np.argmax(probs)==y_test[i][0]:
    correct+=1
  total+=1
print(correct/total)

0.8735384615384615


In [ ]:
# KNN Classifier
import math
def L2distance(sample1, sample2):
  d = 0
  for i in range(len(sample1)):
    d += (sample1[i]-sample2[i])**2
  return math.sqrt(d)

def L1distance(sample1, sample2):
  d=0
  for i in range(len(sample1)):
    d += abs(sample1[i]-sample2[i])
  return d

def Linfdistance(sample1, sample2):
  d = 0
  arr = []
  for i in range(len(sample1)):
    arr.append(abs(sample1[i]-sample2[i]))
  d = max(arr)
  return d

def KNN(k):
  correct = 0
  total = 0
  for i in range(len(X_test)):
    sample1 = X_test[i]
    distances = []
    for sample2 in X_train:
      distances.append(L2distance(sample1, sample2))
    sorted_distances = np.argsort(distances)

    labels = [0]*10
    for j in range(min(k, len(X_train))):
      train_label = y_train[sorted_distances[j]][0]
      labels[train_label]+=1

    if np.argmax(labels) == y_test[i]:
      correct+=1
    total+=1
  
  return correct/total

print(KNN(1))


0.9245


In [ ]:
import matplotlib.pyplot as plt
plt.xscale("log")
x = [1,2,4,10,32,4000,8000]
y = [0.93, 0.92, 0.915, 0.9045, 0.90,  0.11, 0.10]
plt.plot(x,y)
plt.ylabel("Accuracy")
plt.xlabel("k")
plt.title("Nearest Neighbor k-values")
plt.savefig("kNN.png")

In [ ]:
import matplotlib.pyplot as plt
x1 = [3500,5000,6500,8000,9500]
x2 = [3500,5000,6500,8000,9500]
y1 = [0.8735, 0.8754, 0.862, 0.86, 0.85]
y2 = [0.9, 0.9154, 0.922, 0.9245, 0.938]

plt.plot(x1,y1, label="Bayes")
plt.plot(x2,y2, label="kNN")
plt.legend()
plt.ylabel("Accuracy")
plt.xlabel("Training Size")
plt.title("Bayes vs kNN")
plt.savefig("BayesvkNN.png")

In [ ]:
import matplotlib.pyplot as plt
x = [1, 2, 10000]
y = [0.9245, 0.917, 0.8995]
plt.xscale("log")
plt.plot(x,y)
plt.ylabel("Accuracy")
plt.xlabel("L")
plt.title("L Selection")
plt.savefig("kNN L.png")